In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
word_dict = set()
for s in train_data['sentence']:
  words = s[0].split()
  for w in words:
    word_dict.add(w)
    
# generate feature matrix
CntVec = CountVectorizer(vocabulary = word_dict, 
             tokenizer = lambda str: str.split(" "))
tr_d = [item for sublist in train_data['sentence'].tolist() for item in sublist] 
te_d = [item for sublist in test_data['sentence'].tolist() for item in sublist] 
X_train = CntVec.fit_transform(tr_d)
X_test = CntVec.fit_transform(te_d)

In [ ]:
### Start

import string
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords.words('english')
nltk.download('wordnet')


def preprocess(X):
    stop_words = set(stopwords.words('english'))
    X['sentence'] = list(map(lambda x:x.lower(), X['sentence']))
    punctuation = set(string.punctuation)
    X['sentence'] = list(map(lambda x:''.join(ch for ch in x if ch not in punctuation), X['sentence']))
    X['sentence'] = list(map(lambda x:' '.join(w for w in x.split(' ') if w not in stop_words), X['sentence']))
    wnl = nltk.stem.WordNetLemmatizer()
    X['sentence'] = list(map(lambda x:' '.join(wnl.lemmatize(w) for w in x.split(' ')), X['sentence']))
